Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [0]:
#  Use a subset of the data where BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS' and the sale price was more than 100 thousand and less than 2 million.

In [0]:
dfs = df[df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS']

In [0]:
condition = dfs['SALE_PRICE'] > 100000

In [0]:
dfs = dfs[condition]

In [0]:
condition2 = dfs['SALE_PRICE'] < 2000000

In [0]:
dfs['SALE_PRICE'].min()

104000

In [0]:
dfs = dfs[condition2]

In [0]:
dfs['SALE_PRICE'].max()

1955000

In [0]:
# Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.

# Converting SALE_DATE to date time format:
dfs['SALE_DATE'] =  pd.to_datetime(dfs['SALE_DATE'])


In [0]:
# Use data from January — March 2019 to train.

train = dfs[(dfs['SALE_DATE'] >= '2019-01-01') & (dfs['SALE_DATE'] < '2019-04-01')]
train = train.sort_values('SALE_DATE')

In [0]:
# Use data from April 2019 to test.

test = dfs[(dfs['SALE_DATE'] >= '2019-04-01') & (dfs['SALE_DATE'] < '2019-05-01')]
test = train.sort_values('SALE_DATE')

In [0]:
target = 'SALE_PRICE'
high_cardinality = ['ADDRESS', 'BUILDING_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_TIME_OF_SALE', 'TAX_CLASS_AT_PRESENT', 'LAND_SQUARE_FEET']
features = train.columns.drop([target] + high_cardinality + ['APARTMENT_NUMBER'] + ['EASE-MENT'] + ['SALE_DATE'])

In [0]:
dfs.columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'APARTMENT_NUMBER', 'ZIP_CODE',
       'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
       'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE',
       'SALE_PRICE', 'SALE_DATE'],
      dtype='object')

In [0]:
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [0]:
# Do one-hot encoding of categorical features.

import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)

In [0]:
X_train = encoder.fit_transform(X_train)

In [0]:
X_train.dtypes

BOROUGH_3                                            int64
BOROUGH_4                                            int64
BOROUGH_5                                            int64
BOROUGH_2                                            int64
BOROUGH_1                                            int64
NEIGHBORHOOD_OTHER                                   int64
NEIGHBORHOOD_FLUSHING-NORTH                          int64
NEIGHBORHOOD_EAST NEW YORK                           int64
NEIGHBORHOOD_BEDFORD STUYVESANT                      int64
NEIGHBORHOOD_FOREST HILLS                            int64
NEIGHBORHOOD_BOROUGH PARK                            int64
NEIGHBORHOOD_ASTORIA                                 int64
BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS      int64
BLOCK                                                int64
LOT                                                  int64
ZIP_CODE                                           float64
RESIDENTIAL_UNITS                                  float

In [0]:
y_train.dtypes

dtype('int64')

In [0]:
X_test = encoder.transform(X_test)

In [0]:
# Do feature selection with SelectKBest.

from sklearn.feature_selection import SelectKBest, f_regression
import warnings
warnings.simplefilter('ignore')

selector = SelectKBest(score_func=f_regression, k=2)

X_train_selected = selector.fit_transform(X_train, y_train)
X_train_selected.shape

(2507, 2)

In [0]:
# Fit a ridge regression model with multiple features. Use the normalize=True parameter 
# (or do feature scaling beforehand — use the scaler's fit_transform method with the train set, and the scaler's transform method with the test set)

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import warnings
warnings.simplefilter('ignore')

for k in range(1, len(X_train.columns)+1):
    print(f'{k} features')
    
    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)

    model = LinearRegression()
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Test Mean Absolute Error: ${mae:,.0f} \n')